# Singular Value Decomposition with Numpy and SciPy - Lab

## Introduction

In this lab, we will build a basic version of low-rank matrix factorization engine for recommending movies in using Numpy and SciPy. The purpose of this lesson to help develop a sound understanding of how this process works in detail, before we implement it in PySpark. We will use a small movie recommendation dataset suitable for such experiments. The system should make not highly accurate, yet meaningful recommendations. 

## Objectives

You will be able to:

- Build a basic recommendation system using MovieLense 1 million movies dataset
- Use Scipy and Numpy to build a recommendation system using matrix factorization. 


## Dataset
For this lab, we will use  a dataset of 1 million movie ratings  available from the [MovieLens](http://grouplens.org/datasets/movielens/) project collected by GroupLens Research at the University of Minnesota. The website offers many versions and subsets of the complete MovieLens dataset. We have downloaded the 1 million movies subset for you and you can find it under the folder `ml-1m`. [Visit this link](http://files.grouplens.org/datasets/movielens/ml-latest-README.html) and also the MovieLens site above to get an understanding of format for included files before moving on:

- ratings.dat
- users.dat
- movies.dat

Let's first read our dataset into pandas dataframe before moving on. 

Our datasets are `.dat` format with features split using the delimiter `'::'`. Perform following tasks:

- Read the files `ratings.dat`, `movies.dat` and `users.dat` using python's `open()`. Use `encoding='latin-1'` for these files
- Split above files on delimiter '::' and create arrays for users , movies and ratings

- Create ratings and movies dataframes from arrays above with columns:
    - `ratings = ['UID', 'MID', 'Rating', 'Time']`
    - `movies = ['MID', 'Title', 'Genre']`
- View the contents of `movies` and `ratings` datasets

*Note: Make sure to change the appropriate datatypes to int (numeric) in these datasets.*

In [14]:
import pandas as pd

# Code here
ratings_raw = open('ml-1m/ratings.dat', encoding = 'latin-1').read()
ratings_lines = ratings_raw.splitlines()
ratings_tuples = [x.split('::') for x in ratings_lines]
ratings_df = pd.DataFrame(ratings_tuples, columns = ['UID', 'MID', 'Rating', 'Time'])
ratings_df = ratings_df.astype('int')
ratings_df.head()

,UID,MID,Rating,Time
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [15]:
movies_raw = open('ml-1m/movies.dat', encoding = 'latin-1').read()
movies_lines = movies_raw.splitlines()
movies_tuples = [x.split('::') for x in movies_lines]
movies_df = pd.DataFrame(movies_tuples, columns = ['MID', 'Title', 'Genre'])
movies_df['MID'] = movies_df['MID'].astype('int')
movies_df.head()

,MID,Title,Genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [11]:
users_raw = open('ml-1m/users.dat', encoding = 'latin-1').read()
users_lines = users_raw.splitlines()
users_tuples = [x.split('::') for x in users_lines]
users_df = pd.DataFrame(users_tuples, columns = ['UserID','Gender','Age','Occupation','Zipcode'])
users_df.head()

,UserID,Gender,Age,Occupation,Zipcode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


## Creating the Utility Matrix 

Matrix factorization, as we saw in the previous lesson, uses a "Utility Matrix" of users x movies. The intersection points between users and movies indicate the ratings users have given to movies. We saw how this is mostly sparse matrix. Here is a quick refresher below:

<img src="utility.jpg" width=400>

Next, our job is to create such a matrix from the ratings table above in order to proceed forward with SVD. 

- Create a utility matrix `A` to contain one row per user and one column per movie. Pivot `ratings` dataframe to achieve this. 

In [38]:
# Create a utility matrix A by pivoting ratings.df
utility = ratings_df.pivot(index = 'UID', columns='MID', values = 'Rating').fillna(0)
print(utility.mean().shape)
utility.head(10)
# Code here

(3706,)


MID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UID,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,4.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0


Finally, let's perform mean normalization on our utility matrix, and save it as a numpy array for decomposition tasks.

In [25]:
# Mean normalize dataframe A to numpy array A_norm
from sklearn.preprocessing import StandardScaler

utility_norm = StandardScaler(with_std = False).fit_transform(utility.values)
utility_norm[0:10]
# Code here 



array([[ 3.57400662, -0.37152318, -0.23874172, ..., -0.03278146,
        -0.02582781, -0.24288079],
       [-1.42599338, -0.37152318, -0.23874172, ..., -0.03278146,
        -0.02582781, -0.24288079],
       [-1.42599338, -0.37152318, -0.23874172, ..., -0.03278146,
        -0.02582781, -0.24288079],
       ...,
       [ 2.57400662, -0.37152318, -0.23874172, ..., -0.03278146,
        -0.02582781, -0.24288079],
       [ 3.57400662, -0.37152318, -0.23874172, ..., -0.03278146,
        -0.02582781, -0.24288079],
       [ 3.57400662,  4.62847682, -0.23874172, ..., -0.03278146,
        -0.02582781, -0.24288079]])

In [43]:
utility.mean().values.shape

(3706,)

## Matrix Factorization with SVD

SVD can help us decomposes the matrix $A$ into the best lower rank approximation of the original matrix $A$. Mathematically, it decomposes $A$ into a two unitary matrices and a diagonal matrix:

$$\begin{equation}
A = U\Sigma V^{T}
\end{equation}$$

A above is users' ratings matrix (utility), $U$ is the user "features" matrix, $\Sigma$ is the diagonal matrix of singular values (essentially weights), and $V^{T}$ is the movie "features" matrix. $U$ and $V^{T}$ are orthogonal, and represent different things. $U$ represents how much users "like" each feature and $V^{T}$ represents how relevant each feature is to each movie.

To get the lower rank approximation, we take these matrices and keep only the top $k$ features, which we think of as the underlying tastes and preferences vectors.

Perform following tasks:
- Import `svds` from Scipy
- decompose A_norm using 50 factors i.e. pass `k=50` argument to `svds()`


In [32]:
from scipy.sparse.linalg import svds

U, sigma, Vt = svds(utility_norm, k = 50)
# Code here 


### Creating diagonal matrix for sigma factors

The sigma above is returned as a 1 dimensional array of latent factor values. As we need to perform matrix multiplication in our next part, lets convert it to a diagonal matrix using `np.diag()`. [Here is an explanation for this](https://math.stackexchange.com/questions/1731183/convert-vector-into-diagonal-matrix)

- Convert sigma factors into a diagonal matrix with `np.diag()`
- Check and confirm the shape of `sigma` befora and after conversion 

In [35]:
import numpy as np

np.diag(sigma).shape
# Code here



(50, 50)

Excellent, We changed sigma from a vector of size fifty to a 2D diagonal matrix of size 50x50. We can now move on to making predictions from our decomposed matrices. 

## Making Predictions from the Decomposed Matrices

Now we have everything required to make movie ratings predictions for every user. We will do it all at once by following the math and matrix multiply $U$, $\Sigma$, and $V^{T}$ back to get the rank $k=50$ __approximation of $A$__. Perform following tasks to achieve this

* Use `np.dot()` to multiply $U,\Sigma, V$
* add the user ratings means back to get the actual __star__ ratings prediction.

In [40]:
utility_recon = np.dot(np.dot(U, np.diag(sigma)),Vt) + utility.mean().values
utility_recon[0:10]
# Code here 



array([[ 4.19121831e+00,  1.82016507e-01, -2.07548935e-01, ...,
         6.12733810e-02,  5.96932726e-02,  1.23638809e-01],
       [ 8.43570186e-01,  9.87085475e-02,  3.61815304e-01, ...,
        -1.08845592e-01, -5.62673859e-02, -1.40766246e-01],
       [ 1.86928915e+00,  4.76704819e-01,  8.57862265e-02, ...,
         3.53134504e-02,  2.99138644e-02, -1.05525641e-01],
       ...,
       [ 8.21407381e-01,  1.26717687e-01,  6.99659564e-01, ...,
        -4.88302235e-02, -1.89914149e-02, -1.94275809e-02],
       [ 3.62608569e+00,  1.22655875e-01,  4.11789046e-02, ...,
         9.04673329e-02,  7.06557766e-04,  6.27973714e-01],
       [ 4.56336686e+00,  2.95640682e+00,  1.24455328e+00, ...,
         2.57398189e-02, -9.52411493e-04,  5.01932304e-03]])

In [45]:
print(utility.mean().values)
np.dot(np.dot(U, np.diag(sigma)),Vt)[0:10]

[1.42599338 0.37152318 0.23874172 ... 0.03278146 0.02582781 0.24288079]


array([[ 2.76522493e+00, -1.89506672e-01, -4.46290657e-01, ...,
         2.84919241e-02,  3.38654580e-02, -1.19241986e-01],
       [-5.82423191e-01, -2.72814631e-01,  1.23073582e-01, ...,
        -1.41627049e-01, -8.20952005e-02, -3.83647041e-01],
       [ 4.43295772e-01,  1.05181640e-01, -1.52955495e-01, ...,
         2.53199342e-03,  4.08604986e-03, -3.48406436e-01],
       ...,
       [-6.04585996e-01, -2.44805492e-01,  4.60917842e-01, ...,
        -8.16116804e-02, -4.48192295e-02, -2.62308376e-01],
       [ 2.20009231e+00, -2.48867303e-01, -1.97562817e-01, ...,
         5.76858760e-02, -2.51212568e-02,  3.85092919e-01],
       [ 3.13737348e+00,  2.58488365e+00,  1.00581156e+00, ...,
        -7.04163809e-03, -2.67802261e-02, -2.37861472e-01]])

#### For a practical system, the value of `k` above would be identified through creating test and training datasets and selecting optimal value of this parameter. We will leave this bit for our detailed experiment in the next lab. 

Here, we'll see how to make recommendations based on `predictions` array created above.



### Making Recommendations 

With the predictions matrix for every user, we can build a function to recommend movies for any user. We need to return the movies with the highest predicted rating that the specified user hasn't already rated. We will also merge in the user information to get a more complete picture of the recommendations. We will also return the list of movies the user has already rated, for the sake of comparison.
- Create a Dataframe from predictions and view contents
- Use column names of `A` as the new names for this dataframe

In [47]:
preds_df = pd.DataFrame(utility_recon, columns = utility.columns, index = utility.index)
# Code here 
preds_df.head()

MID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UID,,,,,,,,,,,,,,,,,,,,,
1,4.191218,0.182017,-0.207549,-0.035829,0.035604,-0.148269,-0.077992,0.166594,-0.052153,-0.111242,...,0.034037,0.010119,0.029505,-0.004786,-0.072557,0.424864,0.161902,0.061273,0.059693,0.123639
2,0.843570,0.098709,0.361815,0.023612,-0.008163,1.277114,0.037697,0.059085,0.158569,1.496888,...,-0.055547,-0.018887,-0.007720,0.059207,-0.019684,0.128052,-0.430085,-0.108846,-0.056267,-0.140766
3,1.869289,0.476705,0.085786,-0.036092,-0.014967,-0.093985,-0.095999,0.102628,0.040342,0.701308,...,0.044017,0.004954,0.019291,0.038394,0.029997,0.134684,0.086811,0.035313,0.029914,-0.105526
4,0.285030,-0.029099,0.012291,0.070299,0.058086,0.296763,-0.065316,0.033233,0.064682,-0.045562,...,0.012469,0.010524,0.002816,0.015336,-0.071262,0.127542,0.060238,-0.019146,0.027241,-0.011854
5,1.463837,-0.011170,-0.048748,0.247499,-0.054597,1.504200,-0.206618,-0.033040,-0.069144,0.451388,...,0.098862,0.013822,-0.019905,-0.031947,-0.059311,-0.001859,0.516486,0.002316,0.110926,0.236974


Now we have a predictions dataframe composed from a reduced factors. We can now can predicted recommendations for every user. We will create a new function `recommender()` as shown below: 

- `recommender()`
    - Inputs: predictions dataframe , chosen UserID, movies dataframe, original ratings df, num_recommendations
    - Outputs: Movies already rated by user, predicted ratings for remaining movies for user

- Get and set of predictions for selected user and sort in descending order
- Get the movies already rated by user and sort them in descending order by rating
- Create a set of recommendations for all movies, not yet rated by the user

In [66]:
type(preds_df.loc[5])

pandas.core.series.Series

In [109]:
def recommender(predictions_df, UID, movies_df, original_ratings_df, num_recommendations=5):
    
    # Get and sort the user's predictions
    user_row = UID - 1 # UID starts at 1, not 0
    sorted_predictions = predictions_df.iloc[user_row].sort_values(ascending=False) 
    
    # Get the original user data and merge in the movie information 
    user_data = original_ratings_df[original_ratings_df.UID == (UID)]
    user_full = user_data.merge(movies_df, how = 'left', left_on = 'MID', right_on = 'MID').sort_values(['Rating'], ascending=False)

    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = movies_df[~movies_df['MID'].isin(user_full['MID'])] \
                      .merge(pd.DataFrame(sorted_predictions).reset_index(), 
                             how = 'left', left_on = 'MID', right_on = 'MID') \
                      .rename(columns = {UID: 'Predictions'}) \
                      .sort_values(['Predictions'], ascending = False) \
                      .iloc[:num_recommendations, :-1]
                    
    print ('User {0} has already rated {1} movies.'.format(UID, user_full.shape[0]))
    print ('Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    return user_full, recommendations

Using above function, we can now get a set of recommendations for any user. 

In [110]:
# Get a list of already reated and recommended movies for a selected user
# Uncomment to run below


rated, recommended = recommender(preds_df, 100, movies_df, ratings_df, 10)

User 100 has already rated 76 movies.
Recommending highest 10 predicted ratings movies not already rated.


In [111]:
# Uncomment to run 

rated.head(10)

,UID,MID,Rating,Time,Title,Genre
1,100,800,5,977593915,Lone Star (1996),Drama|Mystery
63,100,527,5,977594839,Schindler's List (1993),Drama|War
16,100,919,5,977594947,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical
17,100,924,4,977594873,2001: A Space Odyssey (1968),Drama|Mystery|Sci-Fi|Thriller
29,100,969,4,977594044,"African Queen, The (1951)",Action|Adventure|Romance|War
22,100,2406,4,977594142,Romancing the Stone (1984),Action|Adventure|Comedy|Romance
47,100,318,4,977594839,"Shawshank Redemption, The (1994)",Drama
20,100,858,4,977593950,"Godfather, The (1972)",Action|Crime|Drama
49,100,329,4,977594297,Star Trek: Generations (1994),Action|Adventure|Sci-Fi
50,100,260,4,977593595,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi


In [112]:
# Uncomment to run

print ("\nTop Ten recommendations for selected user" )
recommended


Top Ten recommendations for selected user


,MID,Title,Genre
1311,1374,Star Trek: The Wrath of Khan (1982),Action|Adventure|Sci-Fi
1148,1193,One Flew Over the Cuckoo's Nest (1975),Drama
1312,1376,Star Trek IV: The Voyage Home (1986),Action|Adventure|Sci-Fi
285,296,Pulp Fiction (1994),Crime|Drama
570,590,Dances with Wolves (1990),Adventure|Drama|Western
877,912,Casablanca (1942),Drama|Romance|War
1184,1240,"Terminator, The (1984)",Action|Sci-Fi|Thriller
1161,1214,Alien (1979),Action|Horror|Sci-Fi|Thriller
2779,2916,Total Recall (1990),Action|Adventure|Sci-Fi|Thriller
997,1036,Die Hard (1988),Action|Thriller


For above randomly selected user `100`, we can subjectively evaluate that the recommender is doing a decent job. The movies being recommended are quite similar in taste as movies already rated by user. Remember this system is built using only a small subset of the complete MovieLense database which carries potential for significant improvement in predictive performance.  

## Level Up - Optional 

- Run the experiment again using validation testing to identify the optimal value of rank `k`
- Create Test and Train datasets to predict and evaluate the ratings using a suitable method (e.g. RMSE)
- How much of an improvement do you see in recommendations as a result of validation ? 

- Ask other interesting questions
<img src="ideas.jpg" width=400>

## Additional Resources
- https://towardsdatascience.com/how-to-build-a-simple-song-recommender-296fcbc8c85 - A similar system recommending songs to users

- http://surpriselib.com/ Surprise is a Python Scikit is a very popular module for building receommendation systems

## Summary

In this lab, we learned that we can make good recommendations with collaborative filtering methods using latent features from low-rank matrix factorization methods. This technique also scales significantly better to larger datasets. We will next work with a larger MovieLense dataset and using the mapreduce techniques seen in the previous section, we will use PySpark to implement a similar approach using an implementation of matrix factorization called Alternating Least Squares (ALS) method. 